In [1]:
import pandas as pd
import csv
import json
import requests
from datetime import datetime
import itertools

In [2]:
url = 'https://data.parlameter.si/v1/getVotesTableExtended'
data = pd.read_json(url)


In [3]:
# prepare data
coalition = requests.get('https://data.parlameter.si/v1/getCoalitionPGs').json()['coalition']
orgs = requests.get('https://data.parlameter.si/v1/getAllPGsExt/')
VOTE_MAP = {'za': 1,
            'proti': -1,
            'kvorum': 0,
            'ni': 0,
            'ni_poslanec': 0
            }

def toLogic(row):
    """
    voter option to logic value
    """
    return VOTE_MAP[row['option']]

data['logic'] = data.apply(lambda row: toLogic(row), axis=1)
data['option_ni'] = 0
data['option_za'] = 0
data['option_proti'] = 0
data['option_kvorum'] = 0
data.loc[data['option'] == 'ni', 'option_ni'] = 1
data.loc[data['option'] == 'za', 'option_za'] = 1
data.loc[data['option'] == 'proti', 'option_proti'] = 1
data.loc[data['option'] == 'kvorum', 'option_kvorum'] = 1
data['voter_unit'] = 1
data['is_coalition'] = 0
data.loc[data['voterparty'].isin(coalition), 'is_coalition'] = 1


In [4]:
orgs = requests.get('https://data.parlameter.si/v1/getAllPGsExt/').json()
pairs = []
pairs_ids = list(itertools.combinations(orgs.keys(), 2))
pairs_str = [(orgs[i[0]]['acronym'], orgs[i[1]]['acronym'])for i in pairs_ids]

In [5]:
pairs_ids[0][0]

u'109'

In [18]:
all_votes = data[data.option != 'ni'].groupby('vote_id').sum()

In [7]:
def getPercent(a, b, c):
    a = 0 if pd.isnull(a) else a
    b = 0 if pd.isnull(b) else b
    c = 0 if pd.isnull(c) else c
    return max(a, b, c) / float(sum([a, b, c]))*100

In [8]:
prisotna_koalicija = data[data.option != 'ni'].groupby(['vote_id', 'option']).sum()

In [9]:
a=prisotna_koalicija.reset_index()

In [10]:
oppo_pivot = pd.pivot_table(prisotna_koalicija.reset_index(), index=['vote_id'], values='is_coalition', columns=['option'])
all_pivot = pd.pivot_table(prisotna_koalicija.reset_index(), index=['vote_id'], values='voter_unit', columns=['option'])

In [11]:
oppo_pivot['enotnost_koalicije'] = oppo_pivot.apply(lambda row: getPercent(row['za'], row['proti'], row['kvorum']), axis=1)
all_pivot['enotnost_DZ'] = all_pivot.apply(lambda row: getPercent(row['za'], row['proti'], row['kvorum']), axis=1)

In [12]:
#st. aktivnih odstopanj
def getDeviation(row):
    return row['option_za']+row['option_proti']+row['option_kvorum']-(max(row['option_za'],row['option_proti'],row['option_kvorum']))
grpOpts = data[data.option != 'ni'].groupby(['vote_id', 'voterparty']).sum()
grpOpts['act_dev_of_ps'] = grpOpts.apply(lambda row: getDeviation(row), axis=1)
aktDev = grpOpts.reset_index()[['vote_id','act_dev_of_ps']].groupby('vote_id').sum()

In [255]:
grpOpts = data[data.option != 'ni'].groupby(['vote_id', 'voterparty']).sum()
grpOpts
getDeviation(grpOpts.ix[4677].ix[112])

0.0

In [13]:
# podpora opozicije
def getMaxOption(row):
    opts = [row['option_za'],row['option_proti'],row['option_kvorum']]
    opt = opts.index(max(opts))
    opt_str = ''
    if opt == 0:
        opt_str = 'o_option_za'
    elif opt == 1:
        opt_str = 'o_option_proti'
    elif opt == 2:
        opt_str = 'o_option_kvorum'
    coal_sum = row['o_option_za']+row['o_option_proti']+row['o_option_kvorum']
    return row[opt_str] / float(coal_sum) * 100
    
oppoVotes = grpOpts[grpOpts.is_coalition == 0].reset_index().groupby(['vote_id']).sum()[['option_za', 'option_proti', 'option_kvorum']]
oppoVotes = oppoVotes.rename(columns = {'option_za': 'o_option_za', 'option_proti': 'o_option_proti', 'option_kvorum': 'o_option_kvorum'})
voteOfCoal = grpOpts[grpOpts.is_coalition != 0].reset_index().groupby(['vote_id']).sum()[['option_za', 'option_proti', 'option_kvorum']]
oppoSupport = pd.concat([oppoVotes, voteOfCoal], axis=1)
oppoSupport['oppo_support'] = oppoSupport.apply(lambda row: getMaxOption(row), axis=1)

In [14]:
# najpogostejsi glas je enak tudi pri
def getOrgVote(row):
    opts = [row['option_za'],row['option_proti'],row['option_kvorum']]
    return opts.index(max(opts))
    #0 = 'za'
    #1 = 'proti'
    #2 = 'kvorum'
grpOpts['grp_vote'] = grpOpts.apply(lambda row: getOrgVote(row), axis=1)
a=grpOpts[['grp_vote']].reset_index()
a=a.pivot('vote_id', 'voterparty')
a.head()

grp_vote                                                        
voterparty      1    2    3    5    6    7    8   108  109 111  112 124 125
vote_id                                                                    
4548            1.0  2.0  1.0  0.0  0.0  1.0  2.0 NaN  0.0 NaN  1.0 NaN NaN
4549            1.0  2.0  1.0  0.0  0.0  1.0  0.0 NaN  NaN NaN  1.0 NaN NaN
4550            1.0  2.0  1.0  0.0  0.0  1.0  0.0 NaN  0.0 NaN  1.0 NaN NaN
4551            1.0  2.0  1.0  0.0  1.0  1.0  0.0 NaN  0.0 NaN  1.0 NaN NaN
4552            1.0  2.0  1.0  0.0  0.0  1.0  2.0 NaN  0.0 NaN  1.0 NaN NaN

In [20]:
out = None
for ids, strs in zip(pairs_ids, pairs_str):
    try:
        tempa = a['grp_vote'][[int(ids[0])]]
        tempb = a['grp_vote'][[int(ids[1])]]
        tempa = tempa.rename(columns = {int(ids[0]): 'option'})
        tempb = tempb.rename(columns = {int(ids[1]): 'option'})
        if type(out) != type(None):
            out[strs[0] + '+' + strs[1]] = (tempa == tempb)
        else:
            out = (tempa == tempb)
            out = out.rename(columns = {'option': strs[0] + '+' + strs[1]})
            
    except:
        print strs[0] + '+' + strs[1]
    

PS NP+ZAAB
PS NP+PS NP
SMC+ZAAB
SMC+PS NP
DeSUS+ZAAB
DeSUS+PS NP
IMNS+ZAAB
IMNS+PS NP
SDS+ZAAB
SDS+PS NP
ZAAB+SD
ZAAB+NSI
ZAAB+ZL
ZAAB+PS NP
SD+PS NP
NSI+PS NP
ZL+PS NP


In [22]:
all_votes = all_votes.rename(columns = {'voter_unit':'prisotni'})

In [23]:
#all_pivot = all_pivot.reset_index()
#oppo_pivot = oppo_pivot.reset_index()
dates = data[['vote_id', 'date', 'tags', 'text']].drop_duplicates().reset_index().set_index("vote_id")[["date", 'tags', 'text']]
result = pd.concat([all_pivot['enotnost_DZ'], oppo_pivot[['enotnost_koalicije']], all_votes[['prisotni']], dates, aktDev,oppoSupport[['oppo_support']], out], axis=1)

In [24]:
result.head()

,enotnost_DZ,enotnost_koalicije,prisotni,date,tags,text,act_dev_of_ps,oppo_support,PS NP+SMC,PS NP+DeSUS,...,IMNS+SDS,IMNS+SD,IMNS+NSI,IMNS+ZL,SDS+SD,SDS+NSI,SDS+ZL,SD+NSI,SD+ZL,NSI+ZL
vote_id,,,,,,,,,,,,,,,,,,,,,
4548,69.444444,100.000000,72,2016-11-17 20:50:16,Poslanska vprašanja,Predlog sklepa (razprava o odgovoru ministrice...,0.0,4.347826,False,False,...,False,False,False,True,False,True,False,False,False,False
4549,66.666667,95.744681,69,2016-11-17 20:49:32,Poslanska vprašanja,Predlog sklepa (razprava o odgovoru ministra z...,2.0,4.545455,False,False,...,False,False,False,False,False,True,True,False,False,True
4550,69.444444,100.000000,72,2016-11-17 20:46:17,Poslanska vprašanja,Predlog sklepa (razprava o odgovoru ministra z...,0.0,4.347826,False,False,...,False,False,False,False,False,True,True,False,False,True
4551,71.830986,95.918367,71,2016-11-17 20:45:34,Poslanska vprašanja,Predlog sklepa (razprava o odgovoru ministra z...,4.0,18.181818,False,False,...,False,False,False,False,False,False,True,True,False,False
4552,69.014085,100.000000,71,2016-11-17 20:41:31,Poslanska vprašanja,Predlog sklepa (razprava o odgovoru ministrice...,0.0,4.347826,False,False,...,False,False,False,True,False,True,False,False,False,False


In [25]:
all_votes.head()

,id,orgvoter,result,session_id,voter,voterparty,logic,option_ni,option_za,option_proti,option_kvorum,prisotni,is_coalition
vote_id,,,,,,,,,,,,,
4548,29464491,NaN,0.0,401184,14685,435,-35,0,15,50,7,72,49
4549,28243062,NaN,0.0,384468,14585,324,-25,0,21,46,2,69,47
4550,29477451,NaN,0.0,401184,14685,435,-30,0,20,50,2,72,49
4551,29074458,NaN,71.0,395612,14660,430,-34,0,17,51,3,71,49
4552,29080781,NaN,71.0,395612,14597,434,-34,0,15,49,7,71,48


In [26]:
def setUrl(row):
    return 'https://glej.parlameter.si/s/glasovanje-layered-2/' + str(row.name) + '?frame=true&altHeader=true'
result['url'] = result.apply(lambda row: setUrl(row), axis=1)

In [27]:


#out = result[['url', 'date', 'tags', 'text', 'enotnost_koalicije', 'enotnost_DZ', 'prisotni']]
out = result
out = out.T.drop_duplicates().T
out = out.rename(columns = {'date':'datum'})
#out['datum'] = out['datum'].apply(lambda x: x.strftime('%m-%Y'))   

In [28]:
out.to_csv('../editirial.csv', sep=';', encoding='utf-8')


In [29]:
i=result.reset_index()
i[i.vote_id==6775]

,vote_id,enotnost_DZ,enotnost_koalicije,prisotni,date,tags,text,act_dev_of_ps,oppo_support,PS NP+SMC,...,IMNS+SD,IMNS+NSI,IMNS+ZL,SDS+SD,SDS+NSI,SDS+ZL,SD+NSI,SD+ZL,NSI+ZL,url
1781,6775,50.0,68.181818,76,2017-01-25 20:54:41,"Odbor za notranje zadeve, javno upravo in loka...",Zakon o spremembah in dopolnitvah Zakona o lok...,1.0,21.875,False,...,True,True,False,True,True,False,True,False,False,https://glej.parlameter.si/s/glasovanje-layere...


In [232]:
data.groupby('vote_id')

AttributeError: 'DataFrameGroupBy' object has no attribute 'year'

In [33]:
data

,date,id,option,orgvoter,result,session_id,tags,text,vote_id,voter,voterparty,logic,option_x,voter_unit,is_coalition
0,2016-06-21 21:50:30,444729,za,NaN,True,5576,"Odbor za infrastrukturo, okolje in prostor",Uradno prečiščeno besedilo (Pomorskega zakonika),4943,95,7,1,0,1,1
1,2016-06-21 21:50:30,444730,ni,NaN,True,5576,"Odbor za infrastrukturo, okolje in prostor",Uradno prečiščeno besedilo (Pomorskega zakonika),4943,2,5,0,1,1,0
2,2016-06-21 21:50:30,444731,za,NaN,True,5576,"Odbor za infrastrukturo, okolje in prostor",Uradno prečiščeno besedilo (Pomorskega zakonika),4943,3,1,1,0,1,1
3,2016-06-21 21:50:30,444732,za,NaN,True,5576,"Odbor za infrastrukturo, okolje in prostor",Uradno prečiščeno besedilo (Pomorskega zakonika),4943,4,2,1,0,1,0
4,2016-06-21 21:50:30,444733,za,NaN,True,5576,"Odbor za infrastrukturo, okolje in prostor",Uradno prečiščeno besedilo (Pomorskega zakonika),4943,5,3,1,0,1,1
5,2016-06-21 21:50:30,444734,za,NaN,True,5576,"Odbor za infrastrukturo, okolje in prostor",Uradno prečiščeno besedilo (Pomorskega zakonika),4943,7,109,1,0,1,0
6,2016-06-21 21:50:30,444735,za,NaN,True,5576,"Odbor za infrastrukturo, okolje in prostor",Uradno prečiščeno besedilo (Pomorskega zakonika),4943,8,1,1,0,1,1
7,2016-06-21 21:50:30,444736,ni,NaN,True,5576,"Odbor za infrastrukturo, okolje in prostor",Uradno prečiščeno besedilo (Pomorskega zakonika),4943,9,109,0,1,1,0
8,2016-06-21 21:50:30,444737,za,NaN,True,5576,"Odbor za infrastrukturo, okolje in prostor",Uradno prečiščeno besedilo (Pomorskega zakonika),4943,10,5,1,0,1,0
9,2016-06-21 21:50:30,444738,za,NaN,True,5576,"Odbor za infrastrukturo, okolje in prostor",Uradno prečiščeno besedilo (Pomorskega zakonika),4943,11,1,1,0,1,1


In [275]:
out.ix[4572]

enotnost_DZ                                                      78.481
enotnost_koalicije                                                  100
voter_unit                                                           79
datum                                               2016-11-17 17:41:57
tags                             Odbor za finance in monetarno politiko
text                  Zakon o centralnem kreditnem registru - Amandm...
act_dev_of_ps                                                         1
oppo_support                                                     46.875
act_dev_of_ps                                                         1
PS NP+SMC                                                          True
PS NP+DeSUS                                                        True
PS NP+IMNS                                                         True
PS NP+SDS                                                         False
PS NP+SD                                                        